## Model : marketdirection
### Description :
This model uses a Tensorflow neural network to predict the direction of a market in the next Y periods, based on the values of the previous X periods. 

### Model Attributes :
- FFNN
- Boosting
- Re-training of entire network for each additional period

### USP :
- Normalised market data (between 0 and 1) to highlight common patterns at any time scale.
- Utilises similar markets to increase size of training set


In [ ]:
MODEL_ID = "fdbe5895-0327-49d9-83e9-2246dbe1858b"

MARKET1 = "DOW"
MARKET2 = "SPY"

PIPELINE_ID = "marketdirection"

# TODO : Pull out of pipeline config?
##### Specific to the data ##
NUM_FEATURES = (2 * 4) + 1
NUM_LABELS = 2
#############################

In [ ]:
#
# Get dataset from MI API #
#

import pandas
import sys
import gc
import uuid
import numpy as np

#!pip install --upgrade git+https://github.com/cwilko/quantutils.git

import quantutils.dataset.pipeline as ppl
from quantutils.api.bluemix import ObjectStore, Metrics, Logger
from quantutils.api.marketinsights import MarketInsights
import quantutils.model.utils as mlutils
from quantutils.model.ml import Model

metrics = Metrics('cred/metrics_cred.json')
mi = MarketInsights('cred/MIOapi_cred.json')
objStore = ObjectStore('cred/object_storage_cred.json')
log = Logger('MarketInsights-ML','cred/logging_cred.json')

# Logging helper function
tag = lambda x,y : "".join(["(", x, ":", str(y+1), ") "])

CONFIG = mi.get_model(MODEL_ID)
TRN_CNF = CONFIG['training']

mkt1 = mi.get_dataset(MARKET1, PIPELINE_ID)
mkt2 = mi.get_dataset(MARKET2, PIPELINE_ID)

# Interleave (part of the "added insight" for this model)
mkt1, mkt2, isect = ppl.intersect(mkt1,mkt2)
dataset = ppl.interleave(mkt1,mkt2)

# TODO : This should be config, probably as a proportion
TEST_SET_SIZE = 430
TRAINING_SET_SIZE = len(dataset) - TEST_SET_SIZE
WINDOW_SIZE = TRAINING_SET_SIZE

_, test_y = ppl.splitCol(dataset[TRAINING_SET_SIZE:], NUM_FEATURES)

# Create ML model
ffnn = Model(NUM_FEATURES, NUM_LABELS, CONFIG)

print("Done")

In [ ]:
##
## BOOTSTRAP/BOOSTING TRAINING WITH LOO
##

# Train thread id
train_id = str(uuid.uuid1())[:8]
#train_id = "0b4045ec"

log.info("".join(["(", train_id, ")", " Training model: ", CONFIG['model_desc'], "(",MODEL_ID,") , Pipeline: ", PIPELINE_ID]))
                  
try:
    
    testSetIndex = isect[-(TEST_SET_SIZE/2):]
    predictions = np.array([]).reshape(0,2)
    
    existing1 = mi.get_predictions(MARKET1, MODEL_ID)
    existing2 = mi.get_predictions(MARKET2, MODEL_ID)
    existing_predictions = existing1.index.intersection(existing2.index)
    resultIndex = testSetIndex.tz_localize(None).difference(existing_predictions)
    prediction_idx = np.array([testSetIndex.get_loc(idx) for idx in resultIndex])     
    labels_idx = ppl.interleave(pandas.DataFrame(prediction_idx*2), pandas.DataFrame(prediction_idx*2+1)).values.flatten()
    
    for i in prediction_idx:
        print "Training",
        dataIdx = i * 2 + TRAINING_SET_SIZE
        training_set = dataset[dataIdx-WINDOW_SIZE:dataIdx]
        test_set = dataset[dataIdx:dataIdx+2]
        success = False
        prediction = [-1, -1]
        retry = 0
        while ((not success) & (retry<TRN_CNF['training_retries'])):
            try:
                ## CHOOSE BOOTSTRAP OR BOOST
                #results = mlutils.boostingTrain(ffnn, training_set, test_set, TRN_CNF['lamda'], TRN_CNF['iterations'], CONFIG['debug'])
                results = mlutils.bootstrapTrain(ffnn, training_set, test_set, TRN_CNF['lamda'], TRN_CNF['iterations'], TRN_CNF['threshold'], CONFIG['debug'])
                prediction = np.nanmean(results["test_predictions"], axis=0)
                predictions =  np.concatenate([predictions, prediction])    
                success = True
            except ValueError: 
                print "Value error"
                log.error("".join([tag(train_id, i), "ValueError - Retrying..."]))
                retry = retry + 1
        
        if (not success):
            log.error("Failed to train after several retries")
            break
            
        print "."

        # Extract predictions and store them
        p1, p2 = [pandas.DataFrame([mkt], index=testSetIndex[i:i+1]) for mkt in prediction]
        
        mi.put_predictions(p1, MARKET1, MODEL_ID, update=True)
        mi.put_predictions(p2, MARKET2, MODEL_ID, update=True)
        
        if (True):
            log.debug("".join([tag(train_id, i), testSetIndex[i].isoformat(), " ", MARKET1, ": ", str(p1.values[0])]))
            log.debug("".join([tag(train_id, i), testSetIndex[i].isoformat(), " ", MARKET2, ": ", str(p2.values[0])]))
            
        # Progress statistics
        res = mlutils.evaluate(predictions, test_y[labels_idx][:len(predictions)], TRN_CNF['threshold'])
        log.info("".join([tag(train_id, i), str("Results after %d iterations, %.2f precision, %.2f recall at %.2f threshold" % (i+1, res[0], res[1], TRN_CNF['threshold']))]))   
        metrics.send([{'name':'MI.precision', 'value':res[0].tolist()},{'name':'MI.recall', 'value':res[1].tolist()}])

        # Backup precictions to filestore (deprecated)
        x = 1
        for mkt in ppl.deinterleave(pandas.DataFrame(predictions)):
            mkt.index = resultIndex[:len(mkt)]
            mkt.to_csv("results.csv", header=False)
            objStore.put_file('Experiment2', "results.csv", "".join([MODEL_ID, "_", str(x), ".csv"]) )
            x = x + 1

        # Try to free memory
        gc.collect()
except:
    log.error("".join([tag(train_id, i), str("Unexpected error: %s" % sys.exc_info()[0])]))
    raise
    